In [2]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3162990 sha256=e781559e89d3a9a781f3e49de490818d8ee63cee6f2c83e7452740571ad8ad2c
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


# Sistema de recomondacion

### Librerias

In [3]:
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import KNNBasic
from surprise.accuracy import rmse

from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
import plotly.graph_objects as go
import pandas as pd

### Rutas de los archivos

In [4]:
RATINGS = '/content/RATINGS.csv'
ANIMES = '/content/ANIME.csv'

In [5]:
df_animes = pd.read_csv(ANIMES)
df_rating = pd.read_csv(RATINGS)

In [ ]:
df_rating.columns

Index(['user_id', 'anime_id', 'rating'], dtype='object')

# Genracion del modelo

In [6]:
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(df_rating[['user_id', 'anime_id', 'rating']], reader)

trainset, testset = train_test_split(data, test_size=0.2)

model = KNNBasic(sim_options={'user_based': True})
model.fit(trainset)


predictions = model.test(testset)
accuracy = rmse(predictions)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.2949


In [7]:
accuracy

1.2948642181115446

# Prueba del modelo

In [10]:
df_rating.head(1)

,user_id,anime_id,rating
0,1,8074.0,10.0


In [17]:
df_animes['ANIME_ID'].dtype

dtype('int64')

In [19]:
def predict(user_id:int, model:KNNBasic, df_recommendation:pd.DataFrame, column_id:str ,n_sugestion:int = 5) -> pd.DataFrame:
  recommendations = model.get_neighbors(trainset.to_inner_uid(user_id), k=n_sugestion)
  return df_recommendation[df_recommendation[column_id].isin(recommendations)]


In [26]:
predict(3, model, df_animes,'ANIME_ID')

,ANIME_ID,ANIME_NAME,TYPES,EPISODES,URL_IMAGE,DURATION,SCORED,SCORED_BY,RANKS,POPULARITY,FAVORITES,SYNOPSIS,STATUS,START_DATE,FINISH_DATE,ID_CLASIFICATION,ID_SOURCE
729,91,Mobile Suit Gundam Wing: Endless Waltz,OVA,3,https://myanimelist.cdn-dena.com/images/anime/...,30 min. per ep.,7.86,19822,784.0,1992,190,"In the year After Colony 196, one year after t...",Finished Airing,1997-01-25,1997-07-25,0,0
1151,304,Aa! Megami-sama! Movie,Movie,1,https://myanimelist.cdn-dena.com/images/anime/...,1 hr. 46 min.,7.63,18571,1263.0,2007,94,"For centuries, a god named Celestin has been i...",Finished Airing,2000-10-21,2000-10-21,0,0
1324,102,Aishiteruze Baby★★,TV,26,https://myanimelist.cdn-dena.com/images/anime/...,25 min. per ep.,7.55,39320,1481.0,1121,501,Katakura Kippei is in every way a high school ...,Finished Airing,2004-04-03,2004-10-09,0,1
2700,247,Harukanaru Toki no Naka de: Hachiyou Shou,TV,26,https://myanimelist.cdn-dena.com/images/anime/...,24 min. per ep.,7.14,5362,3040.0,2884,108,Akane Motomiya and her friends Tenma and Shimo...,Finished Airing,2004-10-06,2005-03-30,0,2
3886,293,Di Gi Charat,TV,16,https://myanimelist.cdn-dena.com/images/anime/...,4 min. per ep.,6.82,6288,4232.0,3218,74,Having infiltrated Earth with blind ambitions ...,Finished Airing,1999-11-30,1999-12-24,3,0
